Souce:
- https://huggingface.co/learn/cookbook/en/advanced_rag
- https://arc.net/l/quote/vntkseji

- Flare:
  - https://ayushtues.medium.com/flare-advanced-rag-implemented-from-scratch-07ca75c89800

# Assumptions
- the faiss_index embeddings are up to date

In [3]:
import os
from dotenv import load_dotenv
load_dotenv('.env')
hf_api = os.getenv('HF_API')
HUGGINGFACEHUB_API_TOKEN = hf_api

In [3]:
pip install transformers==4.38.0

In [4]:
!pip install -q torch accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.3 MB/s eta 0:00:00


In [5]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=e34b4c3f81cf6c136e17b9c1087784325d08297d693f46620b901f823eacc1a2
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [6]:
!pip install ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s et

In [7]:
# fix colab error: https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [8]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
pd.set_option(
    "display.max_colwidth", None
)

In [ ]:
# Imports

from langchain.docstore.document import Document as LangchainDocument
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# hf imports
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import T5Tokenizer, T5ForConditionalGeneration
import os

# reranking 

from ragatouille import RAGPretrainedModel
from transformers import Pipeline

In [10]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [11]:
%cd drive/MyDrive/ANLP/NLP-RAG/src-rag
!ls

/content/drive/MyDrive/ANLP/NLP-RAG/src-rag
 database.py	 __pycache__    qa.txt	       'RAG T5+ Normal Flare.ipynb'
 evaluation.py	 qa_gen.ipynb   questions.txt   setup.txt


# Specify the models/versions

In [ ]:
# give the paths
QUESTIONS_FILE = 'data/test/questions_webpages.txt'
OUTPUT_FILE = 'system_outputs/webpages.txt'
FAISS_FILE = 'faiss_index_total' # it's actually a folder but whatever

EMBEDDING_MODEL = "thenlper/gte-base" # make sure this matches whatever was used to create the doc embeddings
GENERATOR_MODEL = "google/flan-t5-large"
RERANKER_MODEL = "colbert-ir/colbertv2.0"


RERANKER = RAGPretrainedModel.from_pretrained(RERANKER_MODEL)


### Load your knowledge base

In [14]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
EMBEDDING_MODEL_NAME = EMBEDDING_MODEL
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  #  True for cosine similarity
)

In [ ]:
# create the vectorized knowledge base
KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local(FAISS_FILE, embedding_model)

In [17]:
# initialize the LLM and its tokenizer, we are using Flan T5 Large for this
tokenizer = T5Tokenizer.from_pretrained(GENERATOR_MODEL)
model = T5ForConditionalGeneration.from_pretrained(GENERATOR_MODEL)

# function to get the prediction and scores from the LLM, given a prompt
def get_prediction_and_scores(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs =  model.generate(input_ids, output_scores=True, return_dict_in_generate=True, max_length=100, skip_special_tokens=True)
    generated_sequence = outputs.sequences[0]

    # get the probability scores for each generated token
    transition_scores = torch.exp(model.compute_transition_scores(
        outputs.sequences, outputs.scores, normalize_logits=True
    )[0])
    return tokenizer.decode(generated_sequence), generated_sequence, transition_scores

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## RAG Prompt

In [ ]:
# prompt_in_chat_format = [
#     {
#         "role": "system",
#         "content": """Using the information contained in the context, give a comprehensive answer to the question.
# Respond only to the question asked, response should be concise and relevant to the question.
# Provide the number of the source document when relevant.
# Give very short answers..
# If the answer cannot be deduced from the context, do not give an answer.""",
#     },
#     {
#         "role": "user",
#         "content": """Context:
# {context}
# ---
# Here is the question you need to answer.
# Question: {question}""",
#     },
# ]
# from langchain.prompts import PromptTemplate
# # RAG_PROMPT_TEMPLATE = PromptTemplate(
# #  template=prompt_in_chat_format, input_variables=["context", "question"]
# # )

# RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
#     prompt_in_chat_format, tokenize=False, add_generation_prompt=True)
# print(RAG_PROMPT_TEMPLATE)

In [ ]:
def answer_with_rag_without_flare(
    question: str,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 5,
    num_docs_final: int = 3,
) -> Tuple[str, List[LangchainDocument]]:


    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    input_text = question
    new_input_text = f"Keep your answers short and concise. Given the below context:\n{context}\n\n Answer the following question \n{input_text}\n. If the answer cannot be deduced from the context, say 'I don't know'."

    # Redact an answer
    print("=> Generating answer...")
    generated_sequence, _, _ = get_prediction_and_scores(new_input_text)
    answer = f"{input_text} {generated_sequence}"
    
    answer = answer.replace("<pad>", "").replace("</s>", "").strip()
    return answer, relevant_docs

In [ ]:
# user_query = 'What are the masters programs in LTI?'
# answer, relevant_docs = answer_with_rag_without_flare(
#     user_query, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER
# )

=> Retrieving documents...


/Users/amandali/anaconda3/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


=> Reranking documents...
Your documents are roughly 331.0 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!
Your documents are roughly 331.0 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 1/1 [00:00<00:00,  1.11it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (989 > 512). Running this sequence through the model will result in indexing errors


=> Generating answer...


In [ ]:

# print("==================================Answer==================================")
# print(len(relevant_docs))
# print(f"{answer}")

==================================Answer==================================
3
What are the masters programs in LTI? <pad> Master of Language Technologies</s>


In [ ]:

# print("==================================Source docs==================================")
# for i, doc in enumerate(relevant_docs):
#     print(f"Document {i}------------------------------------------------------------")
#     print(doc)

==================================Source docs==================================
Document 0------------------------------------------------------------
Courses that satisfy LTI Ph.D. degree requirements may also be used to satisfy requirements for

one M.S. degree. The most common choice is the LTI s Master of Language Technologies (MLT)

degree because its requirements are similar (but not identical) to the Ph.D. requirements. Other M.S. degrees within the LTI and outside of the LTI are also possible. LTI Ph.D.  Graduate Student Handbook  Page 20

Students interested in an M.S. degree other than the MLT degree should  discuss their plans with

their Ph.D. advisor due to the additional courses and project work that may be involved.

3.7 Grading and Evaluation

3.7.1 University Policy on Grades

Carnegie Mellon s Grading  policy offers details concerning university grading principles for

students taking courses and covers the specifics of assigning and c hanging grades, grading

options

In [ ]:
# user_query = 'What is the Buggy race schedule this year?'

In [ ]:
# answer, relevant_docs = answer_with_rag_without_flare(
#     user_query, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER
# )

=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 317.2 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


=> Generating answer...


In [ ]:

# print("==================================Answer==================================")
# print(len(relevant_docs))
# print(f"{answer}")

==================================Answer==================================
3
What is the Buggy race schedule this year? <pad> April 13, 2024</s>


In [ ]:

# print("==================================Source docs==================================")
# for i, doc in enumerate(relevant_docs):
#     print(f"Document {i}------------------------------------------------------------")
#     print(doc)

==================================Source docs==================================
Document 0------------------------------------------------------------
Buggy Races Keep Rolling at Carnegie Mellon -

News - Carnegie Mellon University

Carnegie Mellon University

— — —

Buggy Races Keep Rolling at Carnegie Mellon

April 10, 2019

Buggy Races Keep Rolling at Carnegie Mellon

In its 99th year, the tradition is a Spring Carnival treat

By Heidi Opdyke

opdyke(through)andrew.cmu.edu

Media Inquiries

Julie Mattera

Marketing and Communications

jmattera(through)cmu.edu

412-268-2902

Sweepstakes, also known as the

Buggy Races , has come a long way at Carnegie Mellon University. The slick, torpedo-like vessels carrying drivers with nerves of steel are a far cry from the two-man teams that once changed places halfway through a race and rode in everything from rain barrels with bicycle wheels to three-wheeled ash cans 99 years ago.

Today, it takes six people to maneuver the .84 -mile course ar

## Flare
- essentially an extra acceptance step
- </s>
  -  is the seperation token (shows the end of a sentence. When we say that we break the generation)
  -  if you dont want this remove the if statement where we 'break'

In [ ]:
# initialize the LLM and its tokenizer, we are using Flan T5 Large for this
tokenizer = T5Tokenizer.from_pretrained(GENERATOR_MODEL)
model = T5ForConditionalGeneration.from_pretrained(GENERATOR_MODEL)


# function to get the prediction and scores from the LLM, given a prompt
def get_prediction_and_score_flare(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs =  model.generate(input_ids, output_scores=True, return_dict_in_generate=True, max_length=200)
    generated_sequence = outputs.sequences[0]

    # get the probability scores for each generated token
    transition_scores = torch.exp(model.compute_transition_scores(
        outputs.sequences, outputs.scores, normalize_logits=True
    )[0])
    return tokenizer.decode(generated_sequence), generated_sequence, transition_scores

In [ ]:
def answer_with_rag_flare(
    input_text: str,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 5,
    num_docs_final: int = 3,
    threshold = .2
) -> Tuple[str, List[LangchainDocument]]:

    relevant_docs = None
    while True:
        generated_sequence, tokens, scores = get_prediction_and_scores(input_text)
        if torch.min(scores)< threshold:
            confident_tokens = tokens[torch.where(scores>threshold)]
            query = tokenizer.decode(confident_tokens)

            # Optionally rerank results
            if reranker:
                # Gather documents with retriever
                print("=> Retrieving documents...")
                relevant_docs_acquired = knowledge_index.similarity_search(query=input_text, k=num_retrieved_docs)

                relevant_docs = [doc.page_content for doc in relevant_docs_acquired]
                
                relevant_doc_id = [doc.metadata['source'] for doc in relevant_docs_acquired]
                
                relevant_doc_index = [doc.metadata['start_index'] for doc in relevant_docs_acquired]
                
                # limiting to num_docs_final
                relevant_docs = relevant_docs[:num_docs_final]
                relevant_doc_id = relevant_doc_id[:num_docs_final]
                relevant_doc_index = relevant_doc_index[:num_docs_final]

            # Build the final prompt
            context = "\nExtracted documents:\n"
            context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

            new_input_text = f"Answer the user's questions based on the below context. Keep your answers short and concise.\n------------<context>\n{context}\n</context>------------\n<question> Here is the question\n{input_text}\n</question>\n"

            # Redact an answer
            print("=> Generating answer...")
            generated_sequence, seq, _ = get_prediction_and_scores(new_input_text)
            # input_text = f"{input_text} {generated_sequence}"
            if "</s>" in generated_sequence:
                input_text = tokenizer.decode(seq, skip_special_tokens=True)
                break
        else: # tokens are alrady high confidence
            if "</s>" in generated_sequence:
                input_text = tokenizer.decode(tokens, skip_special_tokens=True)
                break 
    answer = input_text
    print(relevant_docs)
    if relevant_docs is None:
      return answer, None
    
    answer = answer.replace("<pad>", "").replace("</s>", "").strip()

    return answer, relevant_docs

In [ ]:
user_query = 'What is the Buggy race schedule this year?'
answer, relevant_docs = answer_with_rag_flare(
    user_query, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER
)

tensor([   0,    3,    7,  152,    3,    9, 6992,   23,   32,    1])
=> Retrieving documents...


/Users/amandali/anaconda3/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


=> Reranking documents...
Your documents are roughly 317.2 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


=> Generating answer...
tensor([    0,  1186, 10670,   460,  2266,     1])
['Buggy Races Keep Rolling at Carnegie Mellon -\n\nNews - Carnegie Mellon University\n\nCarnegie Mellon University\n\n— — —\n\nBuggy Races Keep Rolling at Carnegie Mellon\n\nApril 10, 2019\n\nBuggy Races Keep Rolling at Carnegie Mellon\n\nIn its 99th year, the tradition is a Spring Carnival treat\n\nBy Heidi Opdyke\n\nopdyke(through)andrew.cmu.edu\n\nMedia Inquiries\n\nJulie Mattera\n\nMarketing and Communications\n\njmattera(through)cmu.edu\n\n412-268-2902\n\nSweepstakes, also known as the\n\nBuggy Races , has come a long way at Carnegie Mellon University. The slick, torpedo-like vessels carrying drivers with nerves of steel are a far cry from the two-man teams that once changed places halfway through a race and rode in everything from rain barrels with bicycle wheels to three-wheeled ash cans 99 years ago.\n\nToday, it takes six people to maneuver the .84 -mile course around Schenley Park\'s Flagstaff Hill.\n\

In [ ]:

print("==================================Answer==================================")
print(len(relevant_docs))
# tokenizer.decode("hello",skip_special_tokens=True)
# print(f"{answer}")
print(answer)

==================================Answer==================================
3
April 13, 2024


In [ ]:
# user_query = 'What did the first doctorate graduate from CMU study?'
# answer, relevant_docs = answer_with_rag_flare(
#     user_query, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER
# )

In [ ]:
# print("==================================Answer==================================")
# # print(len(relevant_docs))
# print(f"{answer}")

## Generate Answers

In [ ]:
def generate_answer(question):
    answer, _ = answer_with_rag_flare(
        question, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER
    )
    return answer

# note that this overwrites previously generated answers to the answer file
def generate_answers_all(qfile, afile):
    questions_file = open(qfile, 'r')
    questions = questions_file.readlines()
    ans_file = open(afile, "w+")
    for q in questions:
        ans = generate_answer(q)
        ans_file.write(ans + '\n')
    questions_file.close()
    ans_file.close()


In [ ]:
generate_answers_all(QUESTIONS_FILE, OUTPUT_FILE)
